In [1]:
# running linear regression on the housing dataset

In [76]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression
%matplotlib inline
import matplotlib.pyplot as plt

In [132]:
train = pd.read_csv('clean_data/train_clean.csv', index_col=0)
#test = pd.read_csv('clean_data/test_clean.csv', index_col=0)

In [127]:
train_numeric = train.select_dtypes(include=[np.number])
train_categorical = train.select_dtypes(exclude=[np.number])

In [130]:
train_numeric.corr()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,X3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
MSSubClass,1.000000,-0.215023,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.023573,-0.069836,-0.065649,...,-0.040110,-0.098672,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.084284
LotFrontage,-0.215023,1.000000,0.100739,0.176561,-0.053457,0.036853,0.078686,0.105010,0.076670,-0.009312,...,0.165229,0.201473,-0.016780,0.069605,0.027366,0.023499,0.022969,0.114106,-0.059606,0.209624
LotArea,-0.139781,0.100739,1.000000,0.105806,-0.005636,0.014228,0.013788,0.103321,0.214103,0.111170,...,0.154871,0.180403,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.263843
OverallQual,0.032628,0.176561,0.105806,1.000000,-0.091932,0.572323,0.550684,0.407252,0.239666,-0.059119,...,0.600671,0.562022,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.790982
OverallCond,-0.059316,-0.053457,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.125694,-0.046231,0.040229,...,-0.185758,-0.151521,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.077856
YearBuilt,0.027850,0.036853,0.014228,0.572323,-0.375983,1.000000,0.592855,0.311600,0.249503,-0.049107,...,0.537850,0.478954,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.522897
YearRemodAdd,0.040581,0.078686,0.013788,0.550684,0.073741,0.592855,1.000000,0.176529,0.128451,-0.067759,...,0.420622,0.371600,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.507101
MasVnrArea,0.023573,0.105010,0.103321,0.407252,-0.125694,0.311600,0.176529,1.000000,0.261256,-0.071330,...,0.361945,0.370884,0.159991,0.122528,-0.109907,0.019144,0.062248,0.011928,-0.029512,0.472614
BsmtFinSF1,-0.069836,0.076670,0.214103,0.239666,-0.046231,0.249503,0.128451,0.261256,1.000000,-0.050117,...,0.224054,0.296970,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,0.386420
BsmtFinSF2,-0.065649,-0.009312,0.111170,-0.059119,0.040229,-0.049107,-0.067759,-0.071330,-0.050117,1.000000,...,-0.038264,-0.018227,0.067898,0.003093,0.036543,-0.029993,0.088871,0.041709,0.004940,-0.011378


In [125]:
trainY = train['SalePrice']
TrainX = train.drop(['SalePrice'], axis=1)


In [30]:
lm = LinearRegression()
lm.fit(trainX, trainY)
lm.score(trainX, trainY)

0.9215044908964388

In [86]:
import statsmodels.api as sm
x = sm.add_constant(trainX)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(trainY, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     223.6
Date:                Fri, 19 Jul 2019   Prob (F-statistic):               0.00
Time:                        17:19:41   Log-Likelihood:                -16776.
No. Observations:                1460   AIC:                         3.368e+04
Df Residuals:                    1395   BIC:                         3.403e+04
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 -1.29e+06 

In [88]:
pValues = results.pvalues
print('The significant coefficients')
sig_coefs = pValues[pValues < .05]
sig_coefs

The significant coefficients


const                   3.329847e-17
garagecars              7.914084e-09
wooddecksf              9.512737e-03
screenporch             4.846038e-03
poolarea                2.891367e-03
saletype_fnew           4.057703e-06
garagecond_ffa          3.274755e-04
garagecond_fgd          2.693848e-03
garagecond_fpo          8.192262e-04
garagecond_fta          2.142903e-04
garagequal_ffa          2.979280e-05
garagequal_fgd          2.262561e-04
garagequal_fpo          2.441930e-04
garagequal_fta          4.853302e-05
poolqc_ffa              7.350631e-06
poolqc_fgd              5.323378e-05
x1stflrsf               1.180410e-08
x2ndflrsf               6.389462e-11
lowqualfinsf            3.378453e-03
grlivarea               2.200574e-06
bedroomabvgr            7.158208e-06
kitchenabvgr            7.862147e-05
totrmsabvgrd            5.013399e-04
fireplaces              1.162822e-04
kitchenqual_ex          5.601943e-10
functional_maj1         2.064112e-02
functional_min1         6.077096e-03
f

In [40]:
significant_coefs = sig_coefs.index.tolist()

In [43]:
# reduce model significant_coefs
reduced_trainX = trainX[significant_coefs]
lm_reduced = LinearRegression()
lm_reduced.fit(reduced_trainX, trainY)
lm_reduced.score(reduced_trainX, trainY)

0.9111916982283614

In [67]:
reduced_x = sm.add_constant(reduced_trainX)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
reduced_model = sm.OLS(trainY, reduced_x)
reduced_results = reduced_model.fit()
#print(reduced_results.summary())
reduced_results.rsquared

0.9111916982283614

In [51]:
pValues = reduced_results.pvalues
print('The significant coefficients')
red_sig_coefs = pValues[pValues < .05]

The significant coefficients


In [99]:
TrainX.columns.tolist()

['garagecars',
 'garagearea',
 'wooddecksf',
 'openporchsf',
 'enclosedporch',
 'x3ssnporch',
 'screenporch',
 'poolarea',
 'miscval',
 'paveddrive_fp',
 'paveddrive_fy',
 'saletype_fcon',
 'saletype_fconld',
 'saletype_fconli',
 'saletype_fconlw',
 'saletype_fcwd',
 'saletype_fnew',
 'saletype_foth',
 'saletype_fwd',
 'garagefinish_fnone',
 'garagefinish_frfn',
 'garagefinish_funf',
 'garagecond_ffa',
 'garagecond_fgd',
 'garagecond_fnone',
 'garagecond_fpo',
 'garagecond_fta',
 'garagequal_ffa',
 'garagequal_fgd',
 'garagequal_fnone',
 'garagequal_fpo',
 'garagequal_fta',
 'poolqc_ffa',
 'poolqc_fgd',
 'poolqc_fnone',
 'fence_fgdwo',
 'fence_fmnprv',
 'fence_fmnww',
 'fence_fnone',
 'miscfeature_fnone',
 'miscfeature_fothr',
 'miscfeature_fshed',
 'miscfeature_ftenc',
 'x1stflrsf',
 'x2ndflrsf',
 'lowqualfinsf',
 'grlivarea',
 'bsmtfullbath',
 'bsmthalfbath',
 'fullbath',
 'halfbath',
 'bedroomabvgr',
 'kitchenabvgr',
 'totrmsabvgrd',
 'fireplaces',
 'garageyrblt',
 'heatingqc_ex',
 

In [102]:
# write for loop to run regression, get best features, run regression, get best features, run regression....
significant_coefs = TrainX.columns.tolist()
n = 50
trainY = df['saleprice']
R2 = []
coefs = []

for i in range(0,n):
    trainX = TrainX[significant_coefs] # reset training set
    
    x = sm.add_constant(trainX)
    model = sm.OLS(trainY, x)
    results = model.fit()
    
    R2.append(results.rsquared)
    pValues = results.pvalues
    new_coefs = pValues[pValues < 0.05].index.tolist()[1:len(pValues[pValues < 0.05].index.tolist())]
    coefs.append(new_coefs)
    significant_coefs = new_coefs

for lst in coefs:
    print(len(lst))

65
63
62
61
60
58
57
56
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55


In [103]:
for r in R2:
    print(r)

0.9215044908964388
0.9090457300840689
0.9088017537286811
0.9085781527515674
0.9083312472663346
0.9081843454871421
0.9077898159047355
0.9075437280622722
0.9073116996238344
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918
0.9070886534226918


In [121]:
cormat = pd.DataFrame(TrainX.corr())
cormat[abs(cormat) < .01].count()

garagecars               24
garagearea               24
wooddecksf               18
openporchsf              22
enclosedporch            36
x3ssnporch               86
screenporch              39
poolarea                 95
miscval                 104
paveddrive_fp            62
paveddrive_fy            26
saletype_fcon           145
saletype_fconld          91
saletype_fconli         133
saletype_fconlw         106
saletype_fcwd           125
saletype_fnew            20
saletype_foth           134
saletype_fwd             20
garagefinish_fnone       39
garagefinish_frfn        23
garagefinish_funf        20
garagecond_ffa           51
garagecond_fgd           91
garagecond_fnone         39
garagecond_fpo           85
garagecond_fta           27
garagequal_ffa           46
garagequal_fgd           85
garagequal_fnone         39
                       ... 
bsmtfintype2_GLQ         75
heating_GasW             74
heating_Grav             89
heating_Wall            104
heating_OthW        